In [3]:
# Setup detectron2 logger
import argparse
import os
import random
import shutil
import time
import warnings
import pickle
import numpy as np
import math
import sys
import copy
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
from itertools import compress
from shapely.geometry import Polygon

import torch
import torch.nn as nn
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer,HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer,ColorMode,GenericMask
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator,inference_on_dataset
from detectron2.data import build_detection_test_loader,DatasetMapper,build_detection_train_loader,MetadataCatalog,DatasetCatalog
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

import ray
import time

In [4]:
# setup directory
root = r'/Users/lovely_shufan/'
project_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2022/0_inference_using_MaskRCNN_2021/'
data_dir = project_dir + r'1_tile_png/GA_MINE_2022_imaged_by_Isabella_Wilson/'
output_dir = project_dir + r'2_infer_result/'

model_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/'

## Create a detectron2 config and a detectron2 DefaultPredictor to run inference 

In [5]:
classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']



In [6]:
cfg = get_cfg() # return default configuration
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) # copy config files from open source projects

# training configuration
cfg.DATASETS.TEST=()
cfg.DATALOADER.NUM_WORKERS=2
#cfg.SOLVER.IMS_PER_BATCH=args.batch_size

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE=128 #Number of regions per image used to train RPN. faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES=len(classes)# (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.BACKBONE.FREEZE_AT=2
cfg.SEED=1
cfg.AUG_FLAG=1

# inference configuration
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS=os.path.join(model_dir, "Trainset1_model_best.pth") # path to the best model trained
cfg.MODEL.DEVICE='cpu' # use cpu for inference


inf_metadata = MetadataCatalog.get("inference").set(thing_classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others'])


## do inference on test image

In [7]:
testimg = cv2.imread(os.path.join(data_dir, 
                                  'PI52606_1_ScanRegion1_15360_3840_17920_5760.png'),cv2.IMREAD_COLOR)

In [8]:
predictor = DefaultPredictor(cfg)
outputs = predictor(testimg)

[09/28 10:15:40 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [9]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(testimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]

cv2.imshow('',out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)



-1

In [10]:
# examine inference outputs
clasind = outputs['instances'].get('pred_classes')
allmasks = outputs['instances'].get('pred_masks')
allscores = outputs['instances'].get('scores')

In [12]:
print(clasind.tolist()[0])
#print(allmasks.shape)
print(allscores)

0
tensor([0.9828, 0.8303, 0.8300])


In [51]:
a = [1,2]

In [52]:
b=allscores.tolist()
 
print(a+b)

[1, 2, 0.9827942848205566, 0.8302654027938843, 0.8300332427024841]


In [47]:
b = np.repeat('PI52606_1',5).tolist()
a = a+b
print(a)

[1, 2, 'PI52606_1', 'PI52606_1', 'PI52606_1', 'PI52606_1', 'PI52606_1']


## do inference on all images

In [28]:
for png in os.listdir(data_dir):
    if png.endswith('.png'):
        print(png)
        img = cv2.imread(os.path.join(data_dir, png), cv2.IMREAD_COLOR)
        predictor = DefaultPredictor(cfg)
        output = predictor(img)
        v = Visualizer(img[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.2)
        out = v.draw_instance_predictions(output["instances"].to("cpu"))
        out_img = out.get_image()[:, :, ::-1]
        cv2.imwrite(os.path.join(output_dir, "_".join([png.replace('.png', ''),'inf_result.png'])), out_img)

PI52606_1_ScanRegion1_10240_1920_12800_3840.png
[09/22 15:31:58 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


PI52606_1_ScanRegion1_12800_0_15360_1920.png
[09/22 15:32:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


PI52606_3_ScanRegion9_10240_1920_12800_3840.png
[09/22 15:32:06 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


...

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


PI52606_2_ScanRegion14_10240_5760_12800_7680.png
[09/22 17:25:38 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


PI52606_2_ScanRegion3_0_0_2560_1920.png
[09/22 17:25:41 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
